# ДЗ

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,8)

model = ['KNN','LC','FCN','CNN',"CNN+BN","Resnet","Resnet + aug."]
accuracy = [0.35,0.39,0.52,0.7,0.78,0.82,0.9]
plt.bar(model,accuracy)
plt.title('CIFAR10 accuracy')
plt.xlabel('model')
plt.ylabel('accuracy')
plt.show()

Siamese Net


051/04_051.png	051_forg/01_0120051.PNG	1 (**1 == Forged**)

for CosineEmbedding

1 -> -1

0 -> 1


https://www.kaggle.com/robinreni/signature-classification-using-siamese-pytorch/comments

# Задачи
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">



Кроме классификации CV решает и другие задачи.

Сегментация, детектирование ...

## COCO

Прежде чем говорить о способах их решения разберемся с форматами фходных данных.

Common objects in context

- один из наиболее популярных датасатов содержащий данные для сегментации и детектирования.

- categoryes
- masks
- bounding boxes
- captions
- person_keypoints
...

https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

Для работы с датасетом используется пакет `pycocotools`

В COCO 90 категорий объектов

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch

In [ ]:
from pycocotools.coco import COCO

coco=COCO('annotations/instances_val2017.json')

cat_ids = coco.getCatIds()
print("Categories count",len(cat_ids))
print(cat_ids)

0 - не используется в качестве номера категории. Обычно его используют для обозначения класса фона.

In [ ]:
# display COCO categories and supercategories

cats = coco.loadCats(coco.getCatIds())
num2cat =  {}
print('COCO categories: ')
for cat in cats:
  num2cat[cat['id']] = cat['name']
  print( cat['id'], ":" , cat['name'],end="   " )


Есть так же суперкатегории

In [ ]:
print(cats[2])
print(cats[3])


nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

Датесет большой, поэтому удобно выгружать данные частями

In [ ]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','cat']); # person and cat
imgIds = coco.getImgIds(catIds=catIds );
print("Total images with person and cat ",len(imgIds))
print(imgIds)
img_list = coco.loadImgs(imgIds[0])
img = img_list[0]
print("Image data", img)

Load image

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

Конвертация в PIL формат

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

def coco2pil(url):
  print(url)
  response = requests.get(img['coco_url'])
  return Image.open(BytesIO(response.content))

pil_img = coco2pil(img['coco_url'])
plt.imshow(pil_img)

Информация о разметке

https://cocodataset.org/#format-data


```
  "segmentation" : RLE or [polygon],
  "area" : float,
  "bbox" : [x,y,width,height],
  "iscrowd" : 0 or 1,
```

Полигон это набор координат [x1,y1, x2,y2 ... ]

объект может описываться несколькими полигонами

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'])
anns = coco.loadAnns(annIds)

def dump_anns(anns):
  for i, a in enumerate(anns):
    print(f"#{i}")
    for k in a.keys():
      if k == 'category_id' and num2cat.get(a[k],None):
        print(k,": ",a[k], num2cat[a[k]]) # Show cat. name
      else:
        print(k,": ",a[k])

dump_anns(anns)
coco.showAnns(anns)

Что такое [RLE](https://en.wikipedia.org/wiki/Run-length_encoding) ?

run-length encoding

https://www.youtube.com/watch?v=h6s61a_pqfM

In [ ]:
plt.rcParams["figure.figsize"] = (160,80)
print(catIds)
annIds = coco.getAnnIds(catIds=catIds, iscrowd = True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)

img = coco.loadImgs(anns[0]['image_id'])[0]
I = io.imread(img['coco_url'])
plt.imshow(I); 
coco.showAnns(anns) # People in the stands 
seg = anns[0]['segmentation']

print('Counts',len(seg['counts']))
print('Size',seg['size'])

Как получить маску в виде массива?

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
import numpy as np
annIds = coco.getAnnIds(imgIds=[448263])

anns = coco.loadAnns(annIds)
msk = np.zeros(seg['size'])
for i in range(len(anns)):
  msk += coco.annToMask(anns[i])*20
print(msk.shape)
plt.figure()
plt.imshow(msk)

print(msk)
print(np.unique(msk))

## Semantic segmentation

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-007.png" width="900">

Постановка задачи:

Предсказать класс для каждого пикселя.

Входные данные маска: 

[ x,y - > class_num ] 

Выходные данные маска:

[ x,y - > class_num ] 


###Способы решения

**а) Наивный.**

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-012.png" width="700">


Скользящим окном пройтись по изображению и предсказать клас для каждого пикселя с учетом его соседей.


**б) Разумный**


Убрать линейный слой в конце сети. По признакам во всех каналах определять класс каждого пикселя.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-015.png" width="700">




<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_15.png"  width="700">

В лекции №8 мы говорили о том что сверту 1x1 можно рассматривать как полносвязанный слой.


Именно так она и будет использоваться при сегментации.

Количество классов будет соответствовать числу каналов.


Проблемы:
- нужно большое рецептивное поле, следовательно много слоев ( L 3х3 conv -> 1+2L receptive field)
- очень медленно на полноразмерных картах активации


**в) Эффективный**


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-017.png" width="700">


Вернмся к традиционной структуре со сжатием пространственных размеров. А после нее добавим разжимающий блок. 

## Автокодировщик

Такая архитектура довольно популярна и применяется не только для сегментации: 


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-07.png" width="700">


- сглаживание шума;
- снижение размерности -> вектор признак
- генерация данных
... 


## Разжимающий(upsample) блок?

### Изменение размеров изображений 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-08.png" width="700">




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-10.png" width="700">

Билинейная интерполяция рассматривает квадрат 2x2 известных пикселя, окружающих неизвестный. В качестве интерполированного значения используется взвешенное усреднение этих четырёх пикселей.


Бикубическая интерполяция идёт на один шаг дальше билинейной, рассматривая массив из 4x4 окружающих пикселей — всего 16. Поскольку они находятся на разных расстояниях от неизвестногопикселя, ближайшие пиксели получают при расчёте больший вес.



С картами признаков можно обращаться так же как и с пикселями. Для этого в Pytorchществует метод Upsample

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-11.png" width="700">

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt

def upsample( pil, mode='nearest' ):
  tensor = TF.to_tensor(pil)
  upsampler = nn.Upsample(scale_factor=2, mode='nearest')
  tensor_128 = upsampler(tensor.unsqueeze(0))
  im_128 = TF.to_pil_image(tensor_128.squeeze()).convert("RGB")
  fig = plt.figure()
  fig.suptitle(mode)
  plt.imshow(im_128)


man_with_cat = coco2pil('http://images.cocodataset.org/val2017/000000223747.jpg')
pil_64 = man_with_cat.resize((64,64))
plt.figure()
plt.imshow(pil_64)


upsample( pil_64, mode='nearest' )
upsample( pil_64, mode='bilinear' )
upsample( pil_64, mode='bicubic' )


### Bed of Nails


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-12.png" width="700">

Способ восстановления размерности когда элементы из начальной карты признаков копируются без изменения, а новые ячейки  вокруг них заполняются нулями.

### MaxUnpooling

Разница в том что индексы элементов запоминаются.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-019.png" width="700">

Сохраняем индексы каждого max pooling слоя
При повышении разрешения копируем значения из выхода max pooling слоя с учетом запомненных индексов


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-14.png" width="700">

https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt

def tensor_show( tensor,title = ''):
  im = TF.to_pil_image(tensor.squeeze()).convert("RGB")
  fig = plt.figure()
  fig.suptitle(title + str(im.size))
  plt.imshow(im)

pool = nn.MaxPool2d(kernel_size = 2, return_indices = True) # False by default
unpool = nn.MaxUnpool2d(kernel_size = 2)

pil = coco2pil('http://images.cocodataset.org/val2017/000000223747.jpg')
fig = plt.figure()
fig.suptitle('original ' +str(pil.size))
plt.imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Initial shape",tensor.shape)

# Downsample
tensor_half_res, indexes1  = pool(tensor)
print("Indexes shape",indexes1.shape)
tensor_show( tensor_half_res,"1/2 down")


tensor_q_res, indexes2  = pool(tensor_half_res)
tensor_show( tensor_q_res, "1/4 down")

# Upsample
tensor_half_res1 = unpool(tensor_q_res,indexes2)

#https://pytorch.org/docs/stable/nn.html#padding-layers
#pad = nn.ZeroPad2d((0,0,0,1))
#tensor_half_res1 = pad(tensor_half_res1)

tensor_show( tensor_half_res1 ,"1/2 up")
print(indexes1.shape)
tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show( tensor_recovered, "full size up")




Зачем нужен pad?


### Transpose convolution

Способы восстановления пространственных размерностей которые мы рассмотрели, не содержали обучаемых параметров.


Обычная свертка:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-16.png" width="700">


Upsample/transpose convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-17.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-15.png" width="700">

https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544


Pytorch
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-19.png" width="700">

https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d

In [ ]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.ConvTranspose2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
input = torch.randn(20, 16, 50, 100)
output = m(input)
# exact output size can be also specified as an argument
input = torch.randn(1, 16, 12, 12)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
h = downsample(input)
print(h.size())
output = upsample(h, output_size=input.size())
print(output.size())


## Unet

Популярная архитектура для сегментации. Изначально создавалась для анализха  медицинских изображений?

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-20.png" width="700">

[U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)

https://github.com/milesial/Pytorch-UNet
https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5


Обратите внимание на серые стрелки на схеме ....



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">



При конкатенации пространственные размеры должны совпадать.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-22.png" width="700">

После upsample блоков ReLU не используется.

https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/


## Несколько классов

Если требуется классифицировать нестколько класов объектов, то в выходной слой нужно добавить соответствующее количество слоев.

# Предобученные модели для сегментации

FCN 

Fully Convolutional Network
для токо что бы не было путаницы с Fully Connected Network
последние именуют MLP (Multi Layer Perceptron)

Usage example
https://pytorch.org/hub/pytorch_vision_fcn_resnet101/

https://pytorch.org/vision/stable/models.html#semantic-segmentation

The pre-trained models have been trained on a subset of COCO train2017, on the 20 categories that are present in the Pascal VOC dataset. You can see more information on how the subset has been selected in references/segmentation/coco_utils.py. The classes that the pre-trained model outputs are the following, in order:


['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [ ]:
import torchvision
from PIL import Image
from torchvision import transforms
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classes = ['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

fcn_model = torchvision.models.segmentation.fcn_resnet50(pretrained=True,  num_classes=21)

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #ImageNet
])

input_tensor = preprocess(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))#['out'][0]




Возвращаются 2 массива

* out - at each location, there are unnormalized probabilities corresponding to the prediction of each class

* aux - contains the auxillary loss values per-pixel. In inference mode, output['aux'] is not usefulcontains the auxillary loss values per-pixel. In inference mode, output['aux'] is not useful

In [ ]:
a = torch.tensor([
                  [[1,4],[7,0.99]],
                  [[0.8,1],[9,12]]
                ]
                  )
a
torch.argmax(a, dim=0)


In [ ]:
print(output.keys()) # Ordered dictionary
print("out", output['out'].shape,"Batch, class_num, h, w") 
print("aux", output['aux'].shape,"Batch, class_num, h, w") 
# aux and output['aux'] contains the auxillary loss values per-pixel. In inference mode, output['aux'] is not usefulcontains the auxillary loss values per-pixel. In inference mode, output['aux'] is not useful

#at each location, there are unnormalized probabilities corresponding to the prediction of each class
output_predictions = output['out'][0].argmax(0)  # for first element of batch
print(output_predictions.shape)
print(output_predictions,torch.max(output_predictions))
indexes = output_predictions
for i, cls_name in enumerate(classes):
  #print(cls_name)
  #print(indexes,indexes.shape)
  mask = torch.zeros(indexes.shape)
  mask[indexes == i] = 255 
  #print(i,class_prob.shape)
  #mask = class_prob.byte()
  #print(mask,mask.shape,torch.max(mask))
  fig = plt.figure()
  fig.suptitle(cls_name)
  plt.imshow(mask)
  #color = i *10
  #break;
  #at each location, there are unnormalized probabilities corresponding to the prediction of each class
  output_predictions += class_prob.argmax(0) * color

#print(output_predictions)
#plt.imshow(output_predictions)

In [ ]:
# create a color pallette, selecting a color for each class
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# plot the semantic segmentation predictions of 21 classes in each color
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(pil_img.size)
r.putpalette(colors)

import matplotlib.pyplot as plt
plt.imshow(r)
# plt.show()

DeepLab

# DeepLab


https://pytorch.org/vision/stable/models.html#deeplabv3

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-24.png" width="700">

## Spatial pyramid pooling layer


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-23.png" width="700">

В pytorch нет отдельного модуля, но результат может быть получени применением нескольких AdaptiveMaxPool2d с разными размерами выходов ...

In [ ]:
# Demo code

## Atros (Dilated) Convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-26.png" width="700">

In [ ]:
# Atros example

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-27.png" width="700">

In [ ]:
#DeepLab example

# Обучение

Входные данные
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-28.png" width="700">

### IoU - ценка точности


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="900">

In [ ]:
# Sample

### Pixel-wise cross entropy loss


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-29.png" width="700">

In [ ]:
#

### DiceLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-30.png" width="700">

https://www.jeremyjordan.me/semantic-segmentation/

In [ ]:
#

# Детекстирование (Object detection)



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">

Постановка задачи:

- опредлить координаты прямоугольника в котором заключен объект.

Пример из задания для семинара:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-05.png" width="700">

Задачу семантической сегментации мы решали через классификацию. Для детектирования разумно использовать регрессию.

В случает для одного объекта можно обучить модель предсказывать числа:

* координаты центра + ширину и высоту
* координаты правого верхнего и левого нижнего углов
* координаты вершин полигона ...



In [ ]:
from torchvision.models import resnet18
from torch import nn
import torch

resnet_detector = resnet18(pretrained = True)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features,4) # x1,y1 x2,y2

criterion  = nn.MSELoss()

input = torch.rand((1,3,224,224))
target = torch.tensor([[.1,.1,.5,.5]]) # x1,y1 x2,y2 or x,y w,h

output = resnet_detector(input)
loss = criterion(output,target)
print(output)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-07.png" width="700">


На прошлом семинаре мы упоминали про модели которые ищут ключевые точки на лице человека (MTCNN). Можно использовать тот же подход для поиска любых точек.


Начнем с ситуации когда объект один.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-08.png" width="500">





In [ ]:
#

У нас есть предобученный классификатор.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-09.png" width="700">

и мы хотим научить его определять местоположение объекта.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-10.png" width="700">

Для этого можно использовать две лосс функции: одна будет оценивать ошибку классификации другая локализации.

Результаты потребуется каким-то образом объединить. В простейшем случае просто сложить.

 ## Regression loss


 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-11.png" width="700">



 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-12.png" width="700">

MAE vs MSE

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-13.png" width="700">

Huber vs Log-cos



## Multitask loss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-14.png" width="500">


[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)


https://arxiv.org/abs/1705.07115

https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb


## Детектирование нескольких объектов

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-042.gif" width="700">


**a) Наивный способ решения: скользящее окно**

Перебрать все возможные местопоположения объектов и классифицировать фрагменты.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-044.gif" width="700">

- очень долго



**b) Эвристика**

 Выбрать области в которых вероятность нахождения объекта наиболее высока (ROI = regions of interest) и проводить поиск только в них.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-049.png" width="700">





#### Selective search

Один из таких алгоритмов:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-18.png" width="1000">

http://www.huppelen.nl/publications/selectiveSearchDraft.pdf

Возвращает порядка 2000 прямоугольников для изображения.

С таким количеством уже можно работать ...

### RCNN
Region CNN 
Построенна по такому принципу:

- на изображении ищутся ROI 
- для кажого делается resize 
- каждый ROI обрабатывается сверточной сетью
которая предсказывает класс кобъекта который в него попал

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-055.png" width="700">


Кроме класса модель предсказывала смещения для каждого bounding box

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-20.png" width="700">

### NMS

Теперь возникает другая проблема в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box) которые частично прекрывают друг друга.


Что бы избавиться от них используется другой алгоритм
Non maxima suppression

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-22.png" width="700">

Его задача избавиться об bbox которые накладиваются на истинный

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-23.png" width="700">


Для оценки схожести обычно используется метрика IoU
а заначение IoU при котором bbox считаются принадлежащими обному объекту является гиперпараметром (часто 0.5)


Soft NMS
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-24.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-25.png" width="700">


RCNN

Проблеммой описанного выше подхода является скорость.
Так как мы вынужденны применять CNN порядка 2000 раз (в зависимости от эвристики которая генерирует ROI)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-26.png" width="700">

И решением является поиск ROI не на самом изображении а на карте признаком, полочкнной после обработке всего изображения CNN. В таком случае большая часть сверток выполняется только один раз.


Fast <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-27.png" width="700">

Это радикально ускоряет процесс








### ROI Pooling

Появляется новая задача - 'resize' ROI на карте признаков.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-28.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-29.png" width="700">

https://pytorch.org/docs/stable/torchvision/ops.html#torchvision.ops.roi_pool




### ROI Align




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-30.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-31.png" width="700">

In [ ]:
#


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-077.png" width="700">

Скорость работы CNN снизилась и теперь узким местом становится эвристика для поиска ROI

### Faster-RCNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-078.png" width="700">

Идею: пусть сеть сама предсказывает ROI по карте признаков


Для обучения требуется посчитать 4 loss.

### Region proposal network

Карта признаков имеет фиксированные и отноститьно небольшие пространственные размеры (например 20x15)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-34.png" width="700">

Поэтому можно вернуться к идее скользящего окна которая была отвергнута в самом начале.

При этом можно использовать окна нескольких форм

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-35.png" width="700">

Предсказываются два значения:

* вероятность того что в ROI находится объект
* смещения



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-086.png" width="700">



### Two stage detector

Faster RCNN == Two stage detector

<img src ="http://edunet.kea.su/repo/src/
L12_Segmentation_Detection/img/lecture_12-089.png" width="700">

## One Stage detector


<img src ="https://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-1.png" width="700">

YOLO, SSD, RetinaNet

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-2.png" width="700">




https://pytorch.org/vision/stable/models.html#runtime-characteristics

### SSD

[SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/ssd.png" width="700">

* VGG-16 model, pre-trained on ImageNet
* manually defines a collection of aspect ratios to use for the B bounding boxes at each grid cell + offsets (x,y,w,h)
* directly predict the probability that a class is present in a given bounding box.
* class for "background"






### Retina net

[Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)

#### FocalLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-43.png" width="1000">

https://amaarora.github.io/2020/06/29/FocalLoss.html

##Feature pyramyd network

[Feature Pyramid Networks for Object Detection](https://arxiv.org/pdf/1612.03144.pdf)

Это feature extractor для детекторов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">

Так же как и в случае с сегментацией, точность повышается если делать предсказания на картах признаков разных масштабов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn1.jpeg" width="400">

Но первые слои содержат мало семантической информации (только низкоуровневые признаки). Из за этого детектирование(и сегментация) мелких объектов удается хуже.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn2.png" width="400">

идея состоит в том что бы делать предсказание с учетом семантической информации полученной на более глубоких слоях. 


При этом признаки суммируются.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn3.png" width="700">


Затем к новым картам признаков может применяться дополнительная свертка.


На выходе получаем карты признаков P2 - P5 на которых уже предсказываются bounding box.


В случае 2-stage детектора (RCNN) карты подаются на вход RPN 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn5.jpeg" width="700">

А признаки для предсказаний используются из backbone 
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn6.jpeg" width="700">


RetinaNet использует выходы FPN и для предсказаний класса и bbox. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-42.png" width="700">


https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c





##YOLO

*  2016 You Only Look Once: Unified, Real-Time Object Detection. 
* 2017 YOLO9000: Better, Faster, Stronger.
* 2018 YOLOv3: An Incremental Improvement

Joseph Redmon

* April 2020, Alexey Bochkovskiy  “YOLOv4: Optimal Speed and Accuracy of Object Detection
* 9 June 2020 YOLOv5 Glenn Jocher https://github.com/ultralytics/yolov5


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-47.png" width="700">

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
! wget https://github.com/Gan4x4/CV-HSE2019/raw/master/data/2bikes.jpeg

In [ ]:
catIds = coco.getCatIds(catNms=['person','bicycle']); # person and bicycle
imgIds = coco.getImgIds(catIds=catIds );
img_list = coco.loadImgs(imgIds[5])
img = img_list[0]
print("Image data", img)

pil_img = coco2pil(img['coco_url'])
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img['id'])
anns = coco.loadAnns(annIds)
coco.showAnns(anns,draw_bbox = True)


Загрузка модели с Torch Hub



In [ ]:
import torch
# Load model from torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


In [ ]:
from PIL import Image

# Apply yolov5 model
results = model(pil_img)
results.print()
results.save() # image on disk

print(type(results.xyxy),len(results.xyxy) , results.xyxy[0].shape)
print(results.xyxy[0])
results.pandas().xyxy[0] 


In [ ]:
dummy_input = torch.rand((1,3,416,416))
results = model(dummy_input)
print(type(results),len(results))
print(results[0].shape)
print(type(results[1]),len(results[1]))
for e in results[1]:
  print(e.shape)
#print(results[0], results[1].shape) # list of two elements

## Нard Example Mining

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-48.png" width="700">


Online hard example mining
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-49.png" width="700">





<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-50.png" width="700">

[Training Region-based Object Detectors with Online Hard Example Mining](https://arxiv.org/pdf/1604.03540.pdf)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors](https://arxiv.org/pdf/1804.04606.pdf)

# Оценка точности

## COCO mAP

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-51.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-54.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-55.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-56.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-57.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-58.png" width="700">

Конвертация результатов сегментации в COCO формат.

https://www.javaer101.com/en/article/18652684.html

Синхронизируем метки

In [ ]:
pascal2coco = {}
print(classes)
def find_in_dic(dic,val):
  for key in dic.keys():
    if dic.get(key,None) == val:
      return key
  return 0 # Assign missed classes to bg

print(num2cat)
for i in range(1, len(classes)): # Skip BG
  #if cats.get()
  coco_ind = find_in_dic(num2cat,classes[i])
  pascal2coco[i] = coco_ind

print(pascal2coco) 


Create gt file


annIds = coco.getAnnIds(catIds=catIds, iscrowd = True)
anns = coco.loadAnns(annIds[0:1])

In [ ]:
from pycocotools import mask


import numpy as np
from itertools import groupby

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

detection_res = []
for i, cls_name in enumerate(classes):
  binary_mask = torch.zeros(indexes.shape)
  binary_mask[indexes == i] = 1 

  if i > 0 and torch.max(binary_mask) > 0  :
  #print(binary_mask.shape,torch.max(binary_mask))
    uncompressed_rle = binary_mask_to_rle(binary_mask.numpy()) #encoded_gt,
    fortran_gt_binary_mask = np.asfortranarray(binary_mask).astype('uint8')
    #print(fortran_ground_truth_binary_mask)
    encoded_gt = mask.encode(fortran_ground_truth_binary_mask)
    #decoded = mask.decode(encoded_ground_truth)
    #print(decoded)
    bbox = list(mask.toBbox(encoded_ground_truth))
    print(bbox)
  #ground_truth_area = mask.area(encoded_ground_truth)
  #ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)


    detection_res.append({
        'score': 1., #dummy
        'category_id': pascal2coco[i],
        #'segmentation' : uncompressed_rle,
        'bbox': bbox, #[80.0, 6.49, 223.89, 223.36], #
        'image_id': 448263,
        #'iscrowd' : 1
    })

print(detection_res)

In [ ]:
import json 
'''
for anno in anns:
        detection_res.append({
            'score': 1.,
            'category_id': anno['category_id'],
            'bbox': anno['bbox'],
            'image_id': anno['image_id']
        })
'''    
 
with open('seg_gt.json', 'w', encoding='utf-8') as f:
  json.dump(anns, f, ensure_ascii=False, indent=4)

In [ ]:
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    import json
    from tempfile import NamedTemporaryFile
     
    # json file in coco format, original annotation data
    #anno_file = 'annotations/instances_val2017.json'
    #coco_gt = COCO('seg_gt.json')
     
     # Use GT box as prediction box for calculation, the purpose is to get detection_res
    #with open(anno_file, 'r') as f:
    #    json_file = json.load(f)
    #annotations = json_file['annotations']


    #detection_res = []
  
   

    import json
    with open('seg_res.json', 'w', encoding='utf-8') as f:
      json.dump(detection_res, f, ensure_ascii=False, indent=4)
    #tf = file.open('seg_res.json'):
    # Due to subsequent needs, first convert detection_res to binary and then write it to the json file
    #content = json.dumps(detection_res).encode(encoding='utf-8')
    #tf.write(content)
    res_path = tf.name
     
    # loadRes will generate a new COCO type instance based on coco_gt and return
    coco_dt = coco_gt.loadRes('seg_res.json')
    coco_gt = coco.loadRes('seg_gt.json')
  
    cocoEval = COCOeval(coco_gt, coco_dt,'bbox') # 'segm', 'bbox'
    #cocoEval.params.useSegm = True
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
     
    print(cocoEval.stats)

# Instance Segmentation

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-03.png" width="700">

COCO panoptic
https://cocodataset.org/#panoptic-2020

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-096.png" width="700">


https://pytorch.org/vision/stable/models.html#mask-r-cnn


Приме запуска Mask R-CNN есть в документации Pytorch: 

https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html